## importing libraries

In [14]:
# !python -m spacy download de_core_news_sm
# !pip install en_core_web_sm
# !pip install en_core_web_md

# !pip install -U spacy
!python -m spacy download de_core_news_sm
!python -m spacy download de_core_news_md


                                              0.0/14.6 MB ? eta -:--:--
                                              0.2/14.6 MB 3.9 MB/s eta 0:00:04
     -                                        0.6/14.6 MB 5.9 MB/s eta 0:00:03
     --                                       0.9/14.6 MB 6.2 MB/s eta 0:00:03
     ---                                      1.3/14.6 MB 6.6 MB/s eta 0:00:03
     ----                                     1.6/14.6 MB 6.9 MB/s eta 0:00:02
     -----                                    1.9/14.6 MB 6.8 MB/s eta 0:00:02
     ------                                   2.4/14.6 MB 7.2 MB/s eta 0:00:02
     -------                                  2.7/14.6 MB 7.6 MB/s eta 0:00:02
     --------                                 3.0/14.6 MB 7.6 MB/s eta 0:00:02
     ---------                                3.3/14.6 MB 7.3 MB/s eta 0:00:02
     ---------                                3.6/14.6 MB 7.1 MB/s eta 0:00:02
     ----------                               3.7/14.6 MB 6

In [15]:
import spacy
nlp = spacy.load("en_core_web_sm")
nlp.max_length = 18500000

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.